In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import codecs
import datetime as dt
import time
import os
import os.path
import shutil


PATH_DIR_DOWNLOADS = os.path.join(f"c:\\Users\\{os.getlogin( )}\\Downloads")
PATH_DIR_DATA_STORE = os.path.join(f".\\Data_store")

In [104]:
driver = webdriver.Chrome()
driver.get('https://email.volganet.ru/?Skin=Samoware')

# Select the Login box
id_box = driver.find_element(By.NAME, 'username')
# Send Login box information
id_box.send_keys('d_korostelev')

# Find password box
pass_box = driver.find_element(By.NAME, 'Password')
# Send password box information
pass_box.send_keys('Ln##9uFT')
time.sleep(2)

# Find login button and Click 
login_button = driver.find_element(By.NAME, 'login').click()
time.sleep(2)

driver.find_element(By.XPATH, "//*[@id=\"samoware-mail\"]/div[1]/div[5]/ul/li[2]/span/span[1]").click()

find_messages_all = driver.find_elements(By.XPATH, "//*[@class=\"samoware-mail-list__item ng-scope\"]/div[1]/div")
find_messages_ods = [item for item in find_messages_all if "ОДС Главного управления МЧС России по Волгоградской области"in item.text]
time.sleep(1.5)

iframe = driver.find_element(By.XPATH, '//*[@id="samoware-mail"]/div[2]/div[3]/div[4]/div[1]/iframe')
list_download_files = list() 

for item in find_messages_ods:
    driver.find_element(By.XPATH, "//*[@id=\"samoware-mail\"]/div[1]/div[5]/ul/li[2]/span/span[1]").click()
    item.click()
    time.sleep(2)

    driver.switch_to.frame(iframe)

    time_start_download = time.time()
    files_find_messages = driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div/button[1]")
    files_find_messages.click()
 
    time.sleep(4)
    time_final_download = time.time()
    driver.switch_to.default_content()
            
    all_download_files = [[filename, os.stat(os.path.join(PATH_DIR_DOWNLOADS, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DOWNLOADS)]
    download_file = [file for file in all_download_files if file[1] >= time_start_download and  file[1] <= time_final_download]
    list_download_files.extend(download_file)
    
# print(dir(driver.switch_to))
# print(files_find_messages[0].tag_name)
# print(files_find_messages[0].text)
# print(files_find_messages[0].get_attribute("class"))
time.sleep(5)

## Проверка файлов и добавление

In [106]:
# Список всех файлов с временем последней модификации:
# Каждый элемент — это имя файла и время 
all_data_store_files = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE)]
all_data_store_files = sorted(all_data_store_files,  reverse=False, key=lambda x:x[1])

# Список всех файлов с временем последней модификации:
# список файлов и время представлены отдельными списками в списке
list_all_data_store_files = [[item[0] for item in all_data_store_files]]
list_all_data_store_files.append([item[1] for item in all_data_store_files])

for file_download in list_download_files:
    ful_path_download_file = os.path.join(PATH_DIR_DOWNLOADS, file_download[0])
    ful_path_data_store_file =  os.path.join(PATH_DIR_DATA_STORE, file_download[0])
    
    if file_download[0] not in list_all_data_store_files[0]:
        shutil.copyfile(ful_path_download_file, ful_path_data_store_file)
        print(f"File: {ful_path_download_file} added to {PATH_DIR_DATA_STORE}")
    elif file_download[0] in list_all_data_store_files[0] and file_download[1] > list_all_data_store_files[1][list_all_data_store_files[0].index(file_download[0])]:
        shutil.copyfile(os.path.join(PATH_DIR_DOWNLOADS, file_download[0]), os.path.join(PATH_DIR_DATA_STORE, file_download[0]))
        print(f"File: {ful_path_download_file} added to {PATH_DIR_DATA_STORE}")
    else:
        print(f"File: {ful_path_download_file} don't add to {PATH_DIR_DATA_STORE}")

File: c:\Users\AdminOS\Downloads\ИВ-219-22-544 Прогноз ЧС ВО на 07.12.2023.PDF don't add to .\Data_store
File: c:\Users\AdminOS\Downloads\3932-20-3-6 Прогноз ЧС по ЮФО на 06 декабря.pdf don't add to .\Data_store


# Создание html страницы

In [109]:
list_all_data_store_files[0]

['3920-20-3-6 Прогноз ЧС по ЮФО на 05 декабря (УТОЧ).pdf',
 'ИВ-219-22-541 Прогноз ЧС ВО на 05.12.2023.PDF',
 '3910-20-3-6 Прогноз ЧС по ЮФО на 04 декабря.pdf',
 'ИВ-219-22-540 Прогноз ЧС ВО на 04.12.2023.PDF',
 'Информационное донесение ИВ-219-22-539.PDF',
 '3902-20-3-6 Прогноз ЧС по ЮФО на 03 декабря.pdf',
 'ИВ-219-22-538 Прогноз ЧС ВО на 03.12.2023.PDF',
 '4МОД (2).pdf',
 'ИВ-219-22-537 Прогноз ЧС ВО на 02.12.2023.PDF',
 'ИВ-219-22-544 Прогноз ЧС ВО на 07.12.2023.PDF',
 '3932-20-3-6 Прогноз ЧС по ЮФО на 06 декабря.pdf']

In [107]:
pattern_file_name_CHS_UFO = 'Прогноз ЧС по ЮФО'
pattern_file_name_CHS_VO = 'Прогноз ЧС ВО'
pattern_file_name_4MOD = '4МОД'
pattern_file_name_IDIV = 'Информационное донесение'


In [116]:
all_files_name_CHS_UFO = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_CHS_UFO in filename]
all_files_name_CHS_UFO = sorted(all_files_name_CHS_UFO,  reverse=False, key=lambda x:x[1])

all_file_name_CHS_VO = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_CHS_VO in filename]
all_file_name_CHS_VO = sorted(all_file_name_CHS_VO,  reverse=False, key=lambda x:x[1])

all_file_name_4MOD = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_4MOD in filename]
all_file_name_4MOD = sorted(all_file_name_4MOD,  reverse=False, key=lambda x:x[1])

all_file_name_IDIV = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_IDIV in filename]
all_file_name_IDIV = sorted(all_file_name_IDIV,  reverse=False, key=lambda x:x[1])


[['3920-20-3-6 Прогноз ЧС по ЮФО на 05 декабря (УТОЧ).pdf',
  1701716174.4801993],
 ['3910-20-3-6 Прогноз ЧС по ЮФО на 04 декабря.pdf', 1701716187.8113368],
 ['3902-20-3-6 Прогноз ЧС по ЮФО на 03 декабря.pdf', 1701716207.6095128],
 ['3932-20-3-6 Прогноз ЧС по ЮФО на 06 декабря.pdf', 1701859288.9923868]]

In [10]:
files_to_site_list = [f'''<tr>
                <td width="20px">{len(all_data_store_files) - i}.</td>
                <td>{all_data_store_files[i][0]}</td>
                <td><a
                    href="Data_store/{all_data_store_files[i][0]}" target="_blank" download="{all_data_store_files[i][0]}">
                    <img src="image/icon_dowload.png" alt="download" width="20px" height="20px"></a></td>
            </tr>''' for i in range(len(all_data_store_files))]
files_to_site_str = ("\n").join(files_to_site_list)


site_page_str = f"""<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8">
    <title></title>
    <link rel="stylesheet" href="style.css">
</head>

<body>
    <div class="dashboard">
        <h1> ОДС Главного управления МЧС России по Волгоградской области</h1>
        <div class="container">
        <table>
            {files_to_site_str}
        </table>
    </div>

    </div>
</body>

</html>
"""

# Html_file = open("index — копия.html","w")
Html_file = codecs.open("index.htm", "w", "utf-8")
Html_file.write(site_page_str)
Html_file.close()